In [1]:
import pyspark

VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
0,application_1618157387188_0001,pyspark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [9]:
from pyspark.sql import functions as F

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [184]:
from pyspark.sql import SQLContext
from pyspark.sql.types import *
from pyspark.sql.types import IntegerType
from pyspark.sql.functions import lower, col
sqlContext = SQLContext(sc)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [188]:
input_bucket = 's3://final-project-spark-twitter-date'
path_favorites = '/data/favorites.csv'
path_friends = '/data/Friends.csv'
path_retweets = '/data/Retweets.csv'
path_statuses = '/data/statuses_colon.csv'
path_users = '/data/users_colon.csv'

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [204]:
# reading in the different tables
df_favorites = sqlContext.read.format('csv').option("inferSchema", "true").option('header','true').load(input_bucket + path_favorites)
df_friends = sqlContext.read.format('csv').option("inferSchema", "true").option('header','true').load(input_bucket + path_friends)
df_retweets = sqlContext.read.format('csv').option("inferSchema", "true").option('header','true').load(input_bucket + path_retweets)
df_statuses = sqlContext.read.format('csv')\
                    .option('delimiter', ';')\
                    .option("quote", "\"")\
                    .option("escape", "\"")\
                    .option("inferSchema", "true")\
                    .option('header','true')\
                    .option('wholeText', 'true')\
                    .option('multiLine', 'true')\
                    .load(input_bucket + path_statuses)
df_users = sqlContext.read.format('csv') \
                    .option('delimiter', ";")\
                    .option('header', "true")\
                    .option("multiLine", "true")\
                    .option('wholeText', 'true')\
                    .option("inferSchema", "true")\
                    .load(input_bucket + path_users)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [91]:
df_favorites.columns

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

['status_id', 'user_id', 'user_screen_name']

In [205]:
df_retweets.columns

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

['created_date', 'status_id', 'retweeted_status', 'lang', 'retweet_count', 'source', 'text', 'user_id']

In [206]:
df_statuses.columns

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

['created_date', 'favorites_count', 'status_id', 'lang', 'retweet_count', 'source', 'text', 'truncated', 'user_id']

In [207]:
df_friends.columns

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

['user_id', 'user_screen_name', 'friend_id', 'friend_screen_name']

In [208]:
df_users.columns

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

['created_date', 'description', 'favorites_count', 'friends_count', 'user_id', 'listed_count', 'location', 'name', 'screen_name', 'statuses_count', 'url', 'verified']

### Answering analytical questions 

#### Q1: Number of statuses by SpaceX account. Answer: 5102

In [99]:
df_users.columns

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

['created_date,description,favorites_count,friends_count,user_id,listed_count,location,name,screen_name,statuses_count,url,verified']

In [133]:
df_users.filter(F.col("screen_name").isin("SpaceX")).collect()[0]['statuses_count']

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

'5102'

#### Q2: Which users tweet the most? Answer: screen_name is 'morinaga_angel'

In [76]:
cols = ['user_id', 'name', 'screen_name', 'statuses_count']

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [137]:
# cast the statuses count to int
df_users_q2 = df_users.withColumn("statuses_count", df_users["statuses_count"].cast(IntegerType()))

# for when statuses_count are not null, find the the user with the most number of tweets
df_users_q2_filtered = df_users_q2.select(*cols, F.col("statuses_count").cast("int").isNotNull().alias("Value"))\
        .filter(F.col("Value")==True).orderBy("statuses_count", ascending=False).limit(5).show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+---------+----------------+---------------+--------------+-----+
|  user_id|            name|    screen_name|statuses_count|Value|
+---------+----------------+---------------+--------------+-----+
|282021927|            ????| morinaga_angel|       4351485| true|
|232635988|????????????????|   angelus_0923|       2902274| true|
|134971995|           Betty|        missb62|       2758994| true|
|113410319|    Vitor Santos|jornalistavitor|       2383144| true|
|834672494| Jean M. O'Brien|   Oldlady12345|       2230945| true|
+---------+----------------+---------------+--------------+-----+

#### Q3: What are the number of mentions per twitter user excluding any retweets? 

In [147]:
import re

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [194]:
def get_all_mentions(df):
    ''' get all mentions from df_statuses'''
    search_string = r'\b@\w+'
    text_with_mentions = df.filter(df['text'].rlike(search_string)).select('text')
    text_with_mentions_list = [row['text'] for row in text_with_mentions.collect()]
    return [mention for mention in ' '.join(text_with_mentions_list).split() if mention[0]=='@']

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [195]:
all_mentions = get_all_mentions(df_statuses)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [193]:
search_string = r'\b@\w+'
all_mentions = df_statuses.filter(df_statuses['text'].rlike(search_string)).select('text').limit(5).show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+
|                text|
+--------------------+
|We're at #IROS201...|
|#lionking@Sethrog...|
|@squarerootsgrow ...|
|@ComdtUSCG @USCG ...|
|Job Vacancy!Tell ...|
+--------------------+

#### Q4: How many statuses mention "SpaceX"? 

In [209]:
df_statuses.columns

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

['created_date', 'favorites_count', 'status_id', 'lang', 'retweet_count', 'source', 'text', 'truncated', 'user_id']

In [210]:
df_statuses.select('text').filter("lower(text) like '%spacex%'").count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

2885

In [211]:
df_statuses.count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

189666